### Comparação usando ElasticSearch entre endereços tratados (base ouro) e brutos (base CNEFE)

In [3]:
import pandas as pd
import unidecode # manipula strings

# Normalização de texto
def normalize(text):
    if pd.isna(text):
        return ""
    texto = unidecode.unidecode(str(text)).strip().lower()
    return texto

# Usa um dicionário para substituir abreviações
def normalizar_abreviacoes(texto):
    abreviacoes = {
        " av ": " avenida ",  
        " avn ": " avenida ",
        " r ": " rua ",
        " pc ": " praca ",
        " al ": " alameda ",
        " tr ": " travessa ",
        " jd ": " jardim ",
        " vl ": " vila "
    }
    texto = " " + texto + " " 
    for abrev, completo in abreviacoes.items():
        texto = texto.replace(abrev, completo)
    return texto.strip()  

df_bruto = pd.read_csv("../documents/3513801_DIADEMA.csv", sep=";", dtype=str)
df_tratado = pd.read_csv("../documents/cnes_geo_padrao_ouro_diadema.csv", sep=";", dtype=str)

### Unificação dos campos dos endereços para comparação

In [4]:
# Concatenação dos campos da base bruta
df_bruto["endereco_original"] = (
    df_bruto["NOM_TIPO_SEGLOGR"].fillna("") + " " +
    df_bruto["NOM_TITULO_SEGLOGR"].fillna("") + " " +
    df_bruto["NOM_SEGLOGR"].fillna("") + " " +
    df_bruto["NUM_ENDERECO"].fillna("") + " " +
    df_bruto["DSC_LOCALIDADE"].fillna("")
).apply(normalize)

# Concatenação dos campos da base tratada
df_tratado["endereco_original"] = (
    df_tratado["NO_LOGRADO"].fillna("") + " " +
    df_tratado["NU_ENDEREC"].fillna("") + " " +
    df_tratado["NO_BAIRRO"].fillna("")
).apply(normalize)

# Usa a normalização de abreviações
df_bruto["endereco_normalizado"] = df_bruto["endereco_original"].apply(normalizar_abreviacoes)  
df_tratado["endereco_normalizado"] = df_tratado["endereco_original"].apply(normalizar_abreviacoes)  

df_tratado[["endereco_original", "endereco_normalizado"]].head(10)

,endereco_original,endereco_normalizado
0,av antonio piranga 614 centro,avenida antonio piranga 614 centro
1,avenida antonio piranga 700 centro,avenida antonio piranga 700 centro
2,av antonio piranga 614 centro,avenida antonio piranga 614 centro
3,rua dos cariris 195 piraporinha,rua dos cariris 195 piraporinha
4,rua moaciar goulart cunha caldas 111 centro,rua moaciar goulart cunha caldas 111 centro
5,praca rui barbosa 27 piraporinha,praca rui barbosa 27 piraporinha
6,av nossa senhora dos navegantes 288 eldorado,avenida nossa senhora dos navegantes 288 eldorado
7,rua capibaribe 193 jardim campanario,rua capibaribe 193 jardim campanario
8,av piraporinha 1682 piraporinha,avenida piraporinha 1682 piraporinha
9,rua jose bonifacio 1641 serraria,rua jose bonifacio 1641 serraria


In [6]:
df_bruto[["endereco_original", "endereco_normalizado"]].head(10)

,endereco_original,endereco_normalizado
0,passagem espelho magico 5 jardim promissao,passagem espelho magico 5 jardim promissao
1,travessa jubiaba 104 inamar,travessa jubiaba 104 inamar
2,avenida sao jose 132 centro,avenida sao jose 132 centro
3,rua dos angicos 62 eldorado,rua dos angicos 62 eldorado
4,rua poti 152 taboao,rua poti 152 taboao
5,rua antonio goncalves martins 178 conceicao,rua antonio goncalves martins 178 conceicao
6,rua casper libero 95 conceicao,rua casper libero 95 conceicao
7,rua casper libero 95 conceicao,rua casper libero 95 conceicao
8,rua casper libero 45 conceicao,rua casper libero 45 conceicao
9,rua do movimento 22 serraria,rua do movimento 22 serraria


### Busca do endereço mais semelhante usando o ElasticSearch

In [5]:
import urllib3
from elasticsearch import Elasticsearch, helpers
import pandas as pd

# Suppress the InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Connect to Elasticsearch (adjust if your instance is remote or uses auth)
# es = Elasticsearch(
#     "https://localhost:9200",
#     basic_auth=("elastic", "rPUgp9*J_vrc9JVx8xB7"),
#     verify_certs=False  # Ignora verificação de certificado SSL
# )

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "rPUgp9*J_vrc9JVx8xB7"),
    ca_certs="/home/samantha/elasticsearch/certs/http_ca.crt"
)


# Index name
INDEX_NAME = "addresses"

# Read CSV with an "address" column
# df = pd.read_csv("../documents/3513801_DIADEMA.csv", sep=';') 

# Create index with mapping (if not exists)
if not es.indices.exists(index=INDEX_NAME):
    es.indices.create(index=INDEX_NAME, body={
        "mappings": {
            "properties": {
                "address": {"type": "text"}
            }
        }
    })

# Bulk index the documents -> armazena endereço, latitude e longitude do endereço
actions = [
    {
        "_index": INDEX_NAME,
        "_source": {"address": row["endereco_normalizado"], "latitude": row["LATITUDE"], "longitude": row["LONGITUDE"]}
    }
    for _, row in df_bruto.iterrows()
]

helpers.bulk(es, actions)

(185596, [])

In [6]:
actions

[{'_index': 'addresses',
  '_source': {'address': 'passagem  espelho magico 5 jardim promissao',
   'latitude': '-23.696257',
   'longitude': '-46.595646'}},
 {'_index': 'addresses',
  '_source': {'address': 'travessa  jubiaba 104 inamar',
   'latitude': '-23.715747',
   'longitude': '-46.608319'}},
 {'_index': 'addresses',
  '_source': {'address': 'avenida sao jose 132 centro',
   'latitude': '-23.686884',
   'longitude': '-46.626084'}},
 {'_index': 'addresses',
  '_source': {'address': 'rua  dos angicos 62 eldorado',
   'latitude': '-23.714338',
   'longitude': '-46.614912'}},
 {'_index': 'addresses',
  '_source': {'address': 'rua  poti 152 taboao',
   'latitude': '-23.663469',
   'longitude': '-46.615173'}},
 {'_index': 'addresses',
  '_source': {'address': 'rua  antonio goncalves martins 178 conceicao',
   'latitude': '-23.698951',
   'longitude': '-46.624475'}},
 {'_index': 'addresses',
  '_source': {'address': 'rua  casper libero 95 conceicao',
   'latitude': '-23.697749',
   'lo

In [7]:
for sample_address in df_tratado["endereco_normalizado"]:
    # Fuzzy search for similar addresses
    response = es.search(index=INDEX_NAME, query={
        "match": {
            "address": {
                "query": sample_address,
                "fuzziness": "AUTO"
            }
        }
    })

    # Print matches
    print(f"Searching for similar addresses to: {sample_address}")
    for hit in response["hits"]["hits"]:
        print(hit["_source"]["address"], "→ score:", hit["_score"])

Searching for similar addresses to: avenida antonio piranga 614 centro
avenida  antonio piranga 114 centro → score: 18.673851
avenida  antonio piranga 615 centro → score: 18.673851
avenida  antonio piranga 641 centro → score: 18.673851
avenida  antonio piranga 61 centro → score: 17.708897
avenida  antonio piranga 125 centro → score: 14.814036
avenida  antonio piranga 173 centro → score: 14.814036
avenida  antonio piranga 163 centro → score: 14.814036
avenida  antonio piranga 141 centro → score: 14.814036
avenida  antonio piranga 159 centro → score: 14.814036
avenida  antonio piranga 149 centro → score: 14.814036
Searching for similar addresses to: avenida antonio piranga 700 centro
avenida  antonio piranga 700 centro → score: 20.491344
avenida  antonio piranga 100 centro → score: 18.598907
avenida  antonio piranga 703 centro → score: 18.598907
avenida  antonio piranga 125 centro → score: 14.814036
avenida  antonio piranga 173 centro → score: 14.814036
avenida  antonio piranga 163 centr

In [9]:
from geopy.distance import geodesic

# for sample_address in df_tratado["endereco_normalizado"]:
for _, row in df_tratado.iterrows():
    sample_address = row["endereco_normalizado"]
    # Fuzzy search for similar addresses
    response = es.search(
        index=INDEX_NAME,
        query={
            "match": {
                "address": {
                    "query": sample_address,
                    "fuzziness": "AUTO"
                }
            }
        }
    )

    # Print matches
    print(f"\nSearching for similar addresses to: {sample_address} | lat: {row["nova_lat"]}, lon: {row["NU_LONGITU"]}")
    for hit in response["hits"]["hits"]:
        source = hit["_source"]
        address = source.get("address")
        latitude = source.get("latitude")
        longitude = source.get("longitude")
        score = hit["_score"]

        if latitude and longitude:
            # Coordenadas: (latitude, longitude)
            coordenada1 = (row["nova_lat"], row["NU_LONGITU"])  # São Paulo
            coordenada2 = (latitude, longitude)  # Rio de Janeiro

            # Distância em quilômetros
            # distancia_km = geodesic(coordenada1, coordenada2).kilometers

            distancia_m = geodesic(coordenada1, coordenada2).meters
            print(f"{address} → score: {score} | lat: {latitude}, lon: {longitude} | distancia: {distancia_m:.2f} m")
        else:
            print(f"{address} → score: {score} | lat: {latitude}, lon: {longitude}")





Searching for similar addresses to: avenida antonio piranga 614 centro | lat: -23.686369, lon: -46.597688900000001
avenida  antonio piranga 114 centro → score: 18.673851 | lat: -23.686451, lon: -46.623345 | distancia: 2616.86 m
avenida  antonio piranga 615 centro → score: 18.673851 | lat: -23.685955, lon: -46.618457 | distancia: 2118.78 m
avenida  antonio piranga 641 centro → score: 18.673851 | lat: -23.685835, lon: -46.618141 | distancia: 2086.89 m
avenida  antonio piranga 61 centro → score: 17.708897 | lat: -23.686363, lon: -46.623730 | distancia: 2656.11 m
avenida  antonio piranga 125 centro → score: 14.814036 | lat: -23.686490, lon: -46.624587 | distancia: 2743.55 m
avenida  antonio piranga 173 centro → score: 14.814036 | lat: -23.686387, lon: -46.624176 | distancia: 2701.60 m
avenida  antonio piranga 163 centro → score: 14.814036 | lat: -23.686541, lon: -46.624262 | distancia: 2710.44 m
avenida  antonio piranga 141 centro → score: 14.814036 | lat: -23.686542, lon: -46.624409 | di

### Exemplo de código para calcular a distancia entre dois pontos no mapa

Ele leva em consideração a curvatura da Terra

In [10]:
from geopy.distance import geodesic

# Coordenadas: (latitude, longitude)
coordenada1 = (-23.550520, -46.633308)  # São Paulo
coordenada2 = (-22.906847, -43.172896)  # Rio de Janeiro

# Distância em quilômetros
distancia_km = geodesic(coordenada1, coordenada2).kilometers
distancia_m = geodesic(coordenada1, coordenada2).meters
print(f"Distância: {distancia_km:.2f} km")
print(f"Distância: {distancia_m:.2f} m")


Distância: 361.26 km
Distância: 361261.39 m


In [ ]:
# continuar:

# no código acima observe que no df_bruto tem uns endereços que tem varios pontos, exemplo:

# rua  dos cariris 195 piraporinha → score: 19.343824 | lat: -23.689450, lon: -46.584908
# rua  dos cariris 195 piraporinha → score: 19.343824 | lat: -23.690810, lon: -46.584650

# como vou calcular a distancia de df_bruto['LATITUDE', 'LONGITUDE'] a df_tratado['nova_lat', 'NU_LONG'] se em df_bruto eu tenho coordenadas diferentes do mesmo endereço?